# Speech-to-text

## convert speech to text

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
speech_key, service_region = os.getenv('SPEECH_SERVICE_KEY'), "canadacentral"

In [ ]:
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

In [ ]:
audio_file = "./BillGates_2010.wav"

In [ ]:
# Creates an audio configuration that points to an audio file
audio_config = speechsdk.audio.AudioConfig(filename=audio_file)

In [ ]:
# Creates a recognizer with the given settings
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

In [ ]:
print("Recognizing speech from file...")
result = speech_recognizer.recognize_once()

In [ ]:
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Recognized: {}".format(result.text))
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized: {}".format(result.no_match_details))
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech Recognition canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))

In [ ]:
# Prints the recognized text
print(result.text)

## summary the text

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [ ]:
endpoint = os.environ["AZURE_LANGUAGE_ENDPOINT"]
key = os.environ["AZURE_LANGUAGE_KEY"]

In [ ]:
text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

In [ ]:
document = result.text

In [ ]:
poller = text_analytics_client.begin_abstract_summary(document)

abstract_summary_results = poller.result()

for result in abstract_summary_results:
    if result.kind == "AbstractiveSummarization":
        print("Summaries abstracted:")
        [print(f"{summary.text}\n") for summary in result.summaries]
    elif result.is_error is True:
        print("...Is an error with code '{}' and message '{}'".format(
            result.error.code, result.error.message
         ))